# Text Generation

Text generation is an LLM use case where the model creates coherent and contextually relevant content based on a prompt or input. This includes tasks like writing blog posts, articles, stories, emails, or even code. 

It helps automate creative and professional writing, enhances productivity, and supports brainstorming by generating ideas or drafts, making it highly valuable in content creation, marketing, and communication.

## Instructing LLMs To Match Tone

LLMs that generate text are awesome, but what if you want to edit the tone/style it responds with?

We've all seen the [pirate](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html#:~:text=template%20%3D%20%22%22%22Answer%20the%20following%20questions%20as%20best%20you%20can%2C%20but%20speaking%20as%20a%20pirate%20might%20speak.%20You%20have%20access%20to%20the%20following%20tools%3A) examples, but it would be awesome if we could tune the prompt to match the tone of specific people?

Below is a series of techniques aimed to generate text in the tone and style you want. No single technique will likely be *exactly* what you need.

### 4 Levels Of Tone Matching Techniques:
1. **Simple:** As a human, try and describe the tone you would like
2. **Intermediate:** Include your description + examples
3. **AI-Assisted:** Ask the LLM to extract tone, use their output in your next prompt
4. **Technique Fusion:** Combine multiple techniques to mimic tone

**Goal**: Generate tweets mimicking the style of online personalities. You could customize this code to generate emails, chat messages, writing, etc.

First let's import our packages

In [ ]:
# Unzip data folder

import zipfile
with zipfile.ZipFile('../../data.zip', 'r') as zip_ref:
    zip_ref.extractall('..')

In [ ]:
# LangChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

# Environment Variables
import os
from dotenv import load_dotenv

# Twitter
import tweepy

load_dotenv()

True

Set your OpenAI key. You can either put it as an environment variable or in the string below

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

We'll be using `gpt-4` today, but you can swap out for `gpt-3.5-turbo` if you'd like

In [ ]:
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, model_name='gpt-4')

### Method #1: Simple - Describe the tone you would like

Our first method is going to be simply describing the tone we would like.

Let's try a few exmaples

In [ ]:
prompt = """
Please create me a tweet about going to the park and eating a sandwich.
"""

output = llm.predict(prompt)
print (output)

"Sunshine, fresh air, and a scrumptious sandwich in hand 🥪🌳 Just had the perfect afternoon at the park, soaking up nature's beauty while munching on my favorite meal! #ParkPicnic #SandwichLover"


Not bad, but I don't love the emojis and I want it to use more conversational modern language.

Let's try again

In [ ]:
prompt = """
Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Use simple language a 5 year old would understand
"""

output = llm.predict(prompt)
print (output)

Had a fun day at the park today! I played on the swings and ate a yummy sandwich for lunch. I love spending time outside!


Ok cool! The tone has changed. Not bad but now I want it to sound like a specific person. Let's try Bill Gates:

In [ ]:
prompt = """
Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates
"""

output = llm.predict(prompt)
print (output)

There's something truly delightful about spending an afternoon at the park, enjoying a well-crafted sandwich, and contemplating the beauty of nature. It's a simple pleasure that reminds us of the importance of taking a break from our busy lives to appreciate the world around us.


It's ok, I'd give the response a `C+` right now.

Let's give some example tweets so the model can better match tone/style.

`⭐ Important Tip: When you're giving examples, make sure to have the examples the same as the desired output format. Ex: Tweets > Tweets, Email > Email`. Don't do `Tweets > Email`

### Method #2: Intermediate - Specify your tone description + examples

Examples speak a thousand words. Let's pass a few along with our instructions to see how it goes

#### Get a users Tweets

Next let's grab a users tweets. We'll do this in a function so it's easy to pull them later

Since we are live Tweets, you'll need to grather some Twitter api keys. You can get these on the [Twitter Developer Portal](https://developer.twitter.com/en/portal/dashboard). The free tier is fine, but watch out for rate limits.

In [ ]:
# Replace these values with your own Twitter API credentials
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY', 'YourAPIKeyIfNotSet')
TWITTER_API_KEY_SECRET = os.getenv('TWITTER_API_KEY_SECRET', 'YourAPIKeyIfNotSet')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN', 'YourAPIKeyIfNotSet')
TWITTER_ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET', 'YourAPIKeyIfNotSet')

In [ ]:
# We'll query 70 tweets because we end up filtering out a bunch, but we'll only return the top 12.
# We will also only use a subset of the top tweets later
def get_original_tweets(screen_name, tweets_to_pull=70, tweets_to_return=12):
    
    # Tweepy set up
    auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_KEY_SECRET)
    auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth)

    tweets = []
    
    tweepy_results = tweepy.Cursor(api.user_timeline,
                                   screen_name=screen_name,
                                   tweet_mode='extended',
                                   exclude_replies=True).items(tweets_to_pull)
    
    # Run through tweets and remove retweets and quote tweets so we can only look at a user's raw emotions
    for status in tweepy_results:
        if not hasattr(status, 'retweeted_status') and not hasattr(status, 'quoted_status'):
            tweets.append({'full_text': status.full_text, 'likes': status.favorite_count})

    
    # Sort the tweets by number of likes. This will help us short_list the top ones later
    sorted_tweets = sorted(tweets, key=lambda x: x['likes'], reverse=True)

    # Get the text and drop the like count from the dictionary
    full_text = [x['full_text'] for x in sorted_tweets][:tweets_to_return]
    
    # Conver the list of tweets into a string of tweets we can use in the prompt later
    example_tweets = "\n\n".join(full_text)
            
    return example_tweets

Let's grab Bill Gates tweets and use those as examples

In [ ]:
user_screen_name = 'billgates'  # Replace this with the desired user's screen name
users_tweets = get_original_tweets(user_screen_name)

Let's look at a sample of Bill's tweets

In [ ]:
print(users_tweets)

These numbers prove why India plays such a crucial role in the world’s fight to improve health, reduce poverty, prevent climate change, and more. https://t.co/xMpmcoYQhi

Mann ki Baat has catalyzed community led action on sanitation, health, women’s economic empowerment and other issues linked to the Sustainable Development Goals. Congratulations @narendramodi on the 100th episode. https://t.co/yg1Di2srjE

The development of AI is as fundamental as the creation of the microprocessor, the personal computer, the Internet, and the mobile phone. It will change the way people work, learn, travel, get health care, and communicate with each other. https://t.co/uuaOQyxBTl

I just returned from my visit to India, and I can’t wait to go back again. I love visiting India because every trip is an incredible opportunity to learn. Here are some photos from my trip and some of the stories behind them: https://t.co/We6PtJWDnp https://t.co/QxZW7gfUmI

Superintelligent AIs are in our future. Compared to

#### Pass the tweets as examples

In [ ]:
template = """
Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates

% START OF EXAMPLE TWEETS TO MIMIC
{example_tweets}
% END OF EXAMPLE TWEETS TO MIMIC

YOUR TWEET:
"""

prompt = PromptTemplate(
    input_variables=["example_tweets"],
    template=template,
)

final_prompt = prompt.format(example_tweets=users_tweets)

print (final_prompt)


Please create me a tweet about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates

% START OF EXAMPLE TWEETS TO MIMIC
These numbers prove why India plays such a crucial role in the world’s fight to improve health, reduce poverty, prevent climate change, and more. https://t.co/xMpmcoYQhi

Mann ki Baat has catalyzed community led action on sanitation, health, women’s economic empowerment and other issues linked to the Sustainable Development Goals. Congratulations @narendramodi on the 100th episode. https://t.co/yg1Di2srjE

The development of AI is as fundamental as the creation of the microprocessor, the personal computer, the Internet, and the mobile phone. It will change the way people work, learn, travel, get health care, and communicate with each other. https://t.co/uuaOQyxBTl

I just returned from my visit to India, and I can’t wait to go back again. I love visiting India because every trip is an incredible op

In [ ]:
output = llm.predict(final_prompt)
print (output)

A simple pleasure like visiting the park and enjoying a sandwich can remind us of the importance of preserving our environment and supporting local food systems. Let's continue to innovate for a sustainable future.


Wow! Ok now that is starting to get somewhere. Not bad at all! Sounds like Bill is in the room with us now.

Let's see if we can refine it even more.

### Method #3: AI-Assisted: Ask the LLM help with tone descriptions

Turns out I'm not great at describing tone. Examples are a good way to help, but can we do more? Let's find out.

I want to have the model tell me what tone *it* sees, then use that output as an *input* to the final prompt where I ask it to generate a tweet.

Almost like reverse engineering tone.

Why don't I do this all in one step? You likely could, but it would be nice to save this "tone" description for future use. Plus, I don't want the model to take too many logic jumps in a single response.

I first thought, 'well... what are the qualities of tone I should have it describe?'

Then I thought, Greg, c'mon man, you know better than that, see if the LLM has a good sense of what tone qualities there are. Duh.

Let's see what are the qualities of tone we should extract

In [ ]:
prompt = """
Can you please generate a list of tone attributes and a description to describe a piece of writing by?

Things like pace, mood, etc.

Respond with nothing else besides the list
"""

how_to_describe_tone = llm.predict(prompt)
print (how_to_describe_tone)

1. Pace: The speed at which the story unfolds and events occur.
2. Mood: The overall emotional atmosphere or feeling of the piece.
3. Tone: The author's attitude towards the subject matter or characters.
4. Voice: The unique style and personality of the author as it comes through in the writing.
5. Diction: The choice of words and phrases used by the author.
6. Syntax: The arrangement of words and phrases to create well-formed sentences.
7. Imagery: The use of vivid and descriptive language to create mental images for the reader.
8. Theme: The central idea or message of the piece.
9. Point of View: The perspective from which the story is told (first person, third person, etc.).
10. Structure: The organization and arrangement of the piece, including its chapters, sections, or stanzas.
11. Dialogue: The conversations between characters in the piece.
12. Characterization: The way the author presents and develops characters in the story.
13. Setting: The time and place in which the story t

Ok great! Now that we have a solid list of ideas on how to instruct our language model for tone. Let's do some tone extraction!

I found that when I asked the model for a description of the tone it would be passive and noncommittal so I included a line in the prompt about taking an active voice 

In [ ]:
def get_authors_tone_description(how_to_describe_tone, users_tweets):
    template = """
        You are an AI Bot that is very good at generating writing in a similar tone as examples.
        Be opinionated and have an active voice.
        Take a strong stance with your response.

        % HOW TO DESCRIBE TONE
        {how_to_describe_tone}

        % START OF EXAMPLES
        {tweet_examples}
        % END OF EXAMPLES

        List out the tone qualities of the examples above
        """

    prompt = PromptTemplate(
        input_variables=["how_to_describe_tone", "tweet_examples"],
        template=template,
    )

    final_prompt = prompt.format(how_to_describe_tone=how_to_describe_tone, tweet_examples=users_tweets)

    authors_tone_description = llm.predict(final_prompt)

    return authors_tone_description

Let's combine the tone description and examples to see what tone attributes the model assigned to Bill Gates

In [ ]:
authors_tone_description = get_authors_tone_description(how_to_describe_tone, users_tweets)
print (authors_tone_description)

1. Pace: Moderate, allowing for thoughtful reflection on the topics discussed.
2. Mood: Optimistic and enthusiastic, highlighting positive aspects and potential solutions.
3. Tone: Engaging and informative, with a strong emphasis on personal experiences and opinions.
4. Voice: Confident and authoritative, showcasing expertise and passion for the subjects.
5. Diction: Clear and concise, using accessible language to convey complex ideas.
6. Syntax: Straightforward and well-structured sentences, making the content easy to follow.
7. Imagery: Evocative and descriptive, painting vivid pictures of experiences and situations.
8. Theme: Focused on innovation, progress, and the potential for positive change.
9. Point of View: First person, providing a personal perspective on the topics discussed.
10. Structure: Organized and coherent, with a logical flow of ideas and information.
11. Dialogue: Limited, but when present, it is engaging and relevant to the topic.
12. Characterization: Presents in

Great, now that we have Bill Gate's tone style, let's put those tone instructions in with the prompt we had before to see if it helps

In [ ]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below
 - Do not use hashtags or emojis
 - Respond in the tone of Bill Gates

% Description of the authors tone:
{authors_tone_description}

% Authors writing samples
{tweet_examples}

% YOUR TASK
Please create a tweet about going to the park and eating a sandwich.
"""

prompt = PromptTemplate(
    input_variables=["authors_tone_description", "tweet_examples"],
    template=template,
)

final_prompt = prompt.format(authors_tone_description=authors_tone_description, tweet_examples=users_tweets)

In [ ]:
llm.predict(final_prompt)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists..


"I recently took a leisurely stroll through the park, enjoying the beauty of nature and savoring a delicious sandwich. It's moments like these that remind us of the simple pleasures in life and inspire us to continue working towards a brighter future for all. https://t.co/9YzF8KJ6rP"

Hmm, better! Not wonderful.

Let's try out the final approach

### Method 4 - **Technique Fusion:** Combine multiple techniques to mimic tone

After a lot of experimentation I've found the below tips to be helpful

* **Don't reference the word 'tweet' in your prompt** - The model has an extremely strong bias towards what a 'tweet' is an will overload you with hashtags and emojis. Rather call it "a short public statement around 300 characters"
* **Ask the LLM for similar sounding authors** - Whereas model bias on the word 'tweet' (point #1) isn't great, we can use it in our favor. Ask the LLM which authors the writing style sounds like, then ask the LLM to respond like that author. It's not great that the model is basing the tone off *another* person but it's a great 89% solution. I learned of this technique from [Scott Mitchell](https://twitter.com/mitchell360/status/1657909800389464064).
* **Examples should be in the output format you want** - Everyone has a different voice. Twitter voice, email voice etc. Make sure that the examples you feed to the prompt are the same voice as the output you want. Ex: Don't exect a book to be written from twitter examples.
* **Use the Language Model to extract tone** - If you are at a loss for words on how to describe the tone you want, have the language model describe it for you. I found I needed to tell the model to be opinionated, it was too grey-area before.
* **Topics matter** - Have the model propose topics *first*, *then* give you a tweet. Not only is it better to have things the author would actually talk about, but it's also really good to keep the model on track by having it outline the topics *first* then respond

Let's first identify authors the model thinks the example tweets sound like, then we'll reference those later. Keep in mind this isn't a true classification exercise and the point isn't to be 100% correct on similar people, it's to get a reference to who the model *thinks* is similar so we can use that inuition for instructions later.

In [ ]:
def get_similar_public_figures(tweet_examples):
    template = """
    You are an AI Bot that is very good at identifying authors, public figures, or writers whos style matches a piece of text
    Your goal is to identify which authors, public figures, or writers sound most similar to the text below

    % START EXAMPLES
    {tweet_examples}
    % END EXAMPLES

    Which authors (list up to 4 if necessary) most closely resemble the examples above? Only respond with the names separated by commas
    """

    prompt = PromptTemplate(
        input_variables=["tweet_examples"],
        template=template,
    )

    # Using the short list of examples so save on tokens and (hopefully) the top tweets
    final_prompt = prompt.format(tweet_examples=tweet_examples)

    authors = llm.predict(final_prompt)
    return authors


In [ ]:
authors = get_similar_public_figures(users_tweets)
print (authors)

Bill Gates


Ok that's not that exciting! Becuase we used Bill Gates' example tweets. Trust me that it's better with less-known people. We'll try this more later.

At last, the final output. Let's bring it all together in a single prompt. Notice the 2 step process in the "your task" section below

In [ ]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below

% Mimic These Authors:
{authors}

% Description of the authors tone:
{tone}

% Authors writing samples
{example_text}
% End of authors writing samples

% YOUR TASK
1st - Write out topics that this author may talk about
2nd - Write a concise passage (under 300 characters) as if you were the author described above
"""

method_4_prompt_template = PromptTemplate(
    input_variables=["authors", "tone", "example_text"],
    template=template,
)

# Using the short list of examples so save on tokens and (hopefully) the top tweets
final_prompt = method_4_prompt_template.format(authors=authors,
                                               tone=authors_tone_description,
                                               example_text=users_tweets)

In [ ]:
# print(final_prompt) # Print this out if you want to see the full final prompt. It's long so I'll omit it for now

In [ ]:
output = llm.predict(final_prompt)
print (output)

1. Topics that this author may talk about:
- Global health and healthcare innovations
- Education and its impact on society
- Climate change and sustainable development
- Technological advancements, such as artificial intelligence and autonomous vehicles
- Poverty reduction and economic empowerment
- Personal experiences and learnings from travels
- Inspirational stories of individuals making a difference

2. Concise passage as the author:
I recently visited a remarkable school in Kenya, where students are using solar-powered tablets to access quality education. It's inspiring to see how technology can transform lives and create a brighter future for these children.


## Personalized email generation

In [1]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

In [ ]:
# Unzip data folder

import zipfile
with zipfile.ZipFile('../../data.zip', 'r') as zip_ref:
    zip_ref.extractall('..')

In [27]:
openai_api_key = '...'

### Load up your HTML from your company page

In [18]:
def get_company_page(company_path):
    y_combinator_url = f"https://www.ycombinator.com{company_path}"
    
    print (y_combinator_url)

    loader = UnstructuredURLLoader(urls=[y_combinator_url])
    return loader.load()


# Get the data of the company you're interested in
data = get_company_page("/companies/poly")
    
print (f"You have {len(data)} document(s)")

https://www.ycombinator.com/companies/poly
You have 1 document(s)


In [19]:
print (f"Preview of your data:\n\n{data[0].page_content[:30]}")

Preview of your data:

Accelerator

About

Apply

FAQ


In [20]:
# Split up the texts so you don't run into token limits
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 800,
    chunk_overlap  = 0
)

In [21]:
docs = text_splitter.split_documents(data)

print (f"You now have {len(docs)} documents")

You now have 3 documents


### Write your custom prompt templates
These will be used for your specific use case:

1. `map_prompt` will be the prompt that is done on your first pass of your documents
2. `combine_prompt` will be the prompt that is used when you combine the outputs of your map pass

In [22]:
map_prompt = """Below is a section of a website about {prospect}

Write a concise summary about {prospect}. If the information is not about {prospect}, exclude it from your summary.

{text}

% CONCISE SUMMARY:"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text", "prospect"])

In [23]:
combine_prompt = """
Your goal is to write a personalized outbound email from {sales_rep}, a sales rep at {company} to {prospect}.

A good email is personalized and combines information about the two companies on how they can help each other.
Be sure to use value selling: A sales methodology that focuses on how your product or service will provide value to the customer instead of focusing on price or solution.

% INFORMATION ABOUT {company}:
{company_information}

% INFORMATION ABOUT {prospect}:
{text}

% INCLUDE THE FOLLOWING PIECES IN YOUR RESPONSE:
- Start the email with the sentence: "We love that {prospect} helps teams..." then insert what they help teams do.
- The sentence: "We can help you do XYZ by ABC" Replace XYZ with what {prospect} does and ABC with what {company} does 
- A 1-2 sentence description about {company}, be brief
- End your email with a call-to-action such as asking them to set up time to talk more

% YOUR RESPONSE:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["sales_rep", "company", "prospect", \
                                                                         "text", "company_information"])

In [24]:
company_information = """
* RapidRoad helps product teams build product faster
* We have a platform that allows product teams to talk more, exchange ideas, and listen to more customers
* Automated project tracking: RapidRoad could use machine learning algorithms to automatically track project progress, identify potential bottlenecks, and suggest ways to optimize workflows. This could help product teams stay on track and deliver faster results.
* Collaboration tools: RapidRoad could offer built-in collaboration tools, such as shared task lists, real-time messaging, and team calendars. This would make it easier for teams to communicate and work together, even if they are in different locations or time zones.
* Agile methodology support: RapidRoad could be specifically designed to support agile development methodologies, such as Scrum or Kanban. This could include features like sprint planning, backlog management, and burndown charts, which would help teams stay organized and focused on their goals.
"""

### LangChain Magic

In [25]:
llm = OpenAI(temperature=.7, openai_api_key=openai_api_key)

chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=map_prompt_template,
                             combine_prompt=combine_prompt_template,
                             verbose=True
                            )

In [26]:
output = chain({"input_documents": docs, # The seven docs that were created before
                "company": "RapidRoad", \
                "company_information" : company_information,
                "sales_rep" : "Greg", \
                "prospect" : "Poly"
               })



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Below is a section of a website about Poly

Write a concise summary about Poly. If the information is not about Poly, exclude it from your summary.

Accelerator

About

Apply

FAQ

People

YC Blog

Companies

Startup Directory

Top Companies

Founder Directory

Launch YC

Startup Jobs

All Jobs

Engineering

Operations

Marketing

Sales

Career Coaching

Pioneer Program 2023

Startup Job Guide

YC Startup Jobs Blog

Upcoming Events

Startup School

About

Co-Founder Matching

Startup School Blog

Library

SAFE

Resources

Event Calendar

Newsletter

For Investors

Hacker News

Open main menu

Apply for S2023 batch.

Apply

Home

Companies

Poly

Poly

Generate Design Assets in Seconds

Y Combinator LogoS22

Active

Artificial Intelligence

Design Tools

Generative AI

Consumer

API

Company

Jobs

News

https://withpoly.com/

Generate Design Assets in Seconds

% CONCISE SUMMARY:
Prompt after formatting:
Below is

In [17]:
print (output['output_text'])


Dear GitLab Team,

We love that GitLab helps teams collaborate, plan, and manage projects, while also providing visibility and consistent controls for security and compliance. We believe that RapidRoad could help you take your DevOps processes to the next level. 

RapidRoad is a platform that helps product teams build faster. We have tools like automated project tracking, collaboration tools, and agile methodology support that could help your teams stay on track and deliver faster results.

We believe that combining GitLab and RapidRoad would unlock your organization from the constraints of the current toolchain. We would love to set up a time to discuss how we can help each other and how we can help you improve your DevOps processes.

Thank you for your time,

Greg 
Sales Rep at RapidRoad


### Rinse and Repeat: Loop Through Companies

In [18]:
import pandas as pd

In [19]:
df_companies = pd.read_clipboard()

In [20]:
df_companies

,Name,Link
0,Findly,/companies/findly
1,Maya Labs,/companies/maya-labs
2,Phind,/companies/phind
3,Juicebox,/companies/juicebox
4,Penguin AI,/companies/penguin-ai
5,Poly,/companies/poly
6,Olli.ai,/companies/olli-ai


In [21]:
for i, company in df_companies.iterrows():
    print (f"{i + 1}. {company['Name']}")
    page_data = get_company_page(company['Link'])
    docs = text_splitter.split_documents(page_data)

    output = chain({"input_documents": docs, \
                "company":"RapidRoad", \
                "sales_rep" : "Greg", \
                "prospect" : company['Name'],
                "company_information" : company_information
               })
    
    print (output['output_text'])
    print ("\n\n")

1. Findly
https://www.ycombinator.com/companies/findly

Dear Findly Team,

We love that Findly helps teams quickly and accurately access data insights without having to learn SQL or Python. We can help you provide more actionable insights by using our platform to talk more, exchange ideas, and listen to more customers. 

RapidRoad is a platform that helps product teams build product faster. It offers automated project tracking, collaboration tools, and agile methodology support to help teams stay organized and deliver faster results. 

We believe that our platform and your AI chatbot will be a great combination to help teams develop products faster and with actionable insights. Let's talk soon and see how we can work together to make this happen. 

Best, 
Greg 
Sales Rep, RapidRoad



2. Maya Labs
https://www.ycombinator.com/companies/maya-labs

Dear Maya Labs, 

We love that Maya Labs helps teams build custom software in minutes using natural language. We at RapidRoad are dedicated to